In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

# Initiate the browser
browser  = webdriver.Chrome(ChromeDriverManager().install())

# Open the Website
browser.get('https://www.linkedin.com/login/es?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Login info
linkedin_username = 'vallarinopedro@gmail.com'
linkedin_password = 'vV1^7I22l#a%gYW^!Jl1'

#Input password
browser.find_element(By.XPATH,'//*[@id="username"]').send_keys(linkedin_username)
browser.find_element(By.XPATH,'//*[@id="password"]').send_keys(linkedin_password)
browser.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()

browser.get('https://www.linkedin.com/jobs/search/?currentJobId=3222732029&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_WT=2&geoId=92000000&keywords=Data%20Analyst')
number_of_jobs = re.findall(r'\d+', browser.find_element_by_xpath('//*[@id="main"]/div/section[1]/header/div[1]/small').get_attribute('innerText'))
number_of_jobs = ''.join(number_of_jobs)
number_of_jobs = int(number_of_jobs)

time.sleep(5)

from selenium.webdriver.common.keys import Keys

job_titles = browser.find_elements_by_class_name('jobs-search-results__list-item')
browser.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

browser.close()

In [ ]:

'''
i = 2
while i <= int(number_of_jobs/25)+1: 
    i = i + 1


jobs = job_lists.find_elements_by_tag_name('li') # return a list

     try:
     wd.find_element_by_xpath(‘/html/body/main/div/section/button’).click()
     time.sleep(5)
     except:
         pass
         time.sleep(5) '''

In [ ]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

for job in jobs:
    job_id0 = job.get_attribute('data-id')
    job_id.append(job_id0)
 
    job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
 
    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)
 
    location0 = job.find_element_by_css_selector('[class=”job-result-card__location”]').get_attribute('innerText')
    location.append(location0)
 
    date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
 
    job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
    job_link.append(job_link0)